# Gathering Data

In [1]:
from functions import *
from scivol import *
from helpers import *
import ipyvolume as ipv
import numpy as np
import json
import ants
import gzip

In [2]:
proj_root = parent_directory()
print(proj_root)
mri_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
template_folder = "/Users/joachimpfefferkorn/repos/neurovolume/templates"
output_folder = os.path.join(proj_root, "output/")

default_cmap = 'nipy_spectral'

/Users/joachimpfefferkorn/repos/neurovolume


In [3]:
raw_anat_img = ants.image_read(mri_input_filepath)

In [4]:
# [ ] MOVE TO TOP
from notebook_viewer_functions import *

In [5]:
print(raw_anat_img)
explore_3D_vol(raw_anat_img.numpy(), dim="x")
explore_3D_vol(raw_anat_img.numpy(), dim="y")
explore_3D_vol(raw_anat_img.numpy(), dim="z")

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=147, description='slice', max=295), Output()), _dom_classes=('widget-int…

# Skull Stripping
Using ANTs

[this notebook](https://github.com/Angeluz-07/MRI-preprocessing-techniques/blob/main/notebooks/09_brain_extraction_with_template.ipynb) is what we're basing this on

## Load Template and Mask

In [ ]:
mni_template_img = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_template_mask = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")

In [ ]:
print(mni_template_img)
explore_3D_array(mni_template_img.numpy(), cmap=default_cmap)

In [ ]:
print(mni_template_mask)
explore_3D_array(mni_template_mask.numpy(), cmap=default_cmap)

## Register Template and Masks to Raw image
Here we warp the MNI template

In [ ]:
template_warp_to_raw_anat = ants.registration(
    fixed=raw_anat_img,
    moving=mni_template_img, 
    type_of_transform='SyN',
    verbose=True
)

In [ ]:
registered_template_img = template_warp_to_raw_anat['warpedmovout']

The following comparison function asserts that the new `registered_template_img` image has been transformed to the size and shape of the `raw_anat_img`

In [ ]:
explore_3D_array_comparison(
    arr_before=raw_anat_img.numpy(),
    arr_after=registered_template_img.numpy()
)
#I don't love "before" and "after" as the names here as we are really only changing the template
# This is pretty confusing, so just ignore the names for now
# Maybe I'll change this and write my own helper functions

Next, we apply this to the mask so that it fits over the brain in the data we wish to visualize

In [ ]:
brain_mask = ants.apply_transforms(
    fixed=template_warp_to_raw_anat['warpedmovout'],
    moving=mni_template_mask,
    transformlist=template_warp_to_raw_anat['fwdtransforms'],
    interpolator='nearestNeighbor',
    verbose=True
)

In [ ]:
explore_3D_array_with_mask_contour(raw_anat_img.numpy(), brain_mask.numpy())

Let's fine tune the mask a bit

In [ ]:
brain_mask_dilated = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')

In [ ]:
explore_3D_array_with_mask_contour(raw_anat_img.numpy(), brain_mask_dilated.numpy())

# Mask the Image and Create Sections

In [ ]:
brain_anat = ants.mask_image(raw_anat_img, brain_mask_dilated)

In [ ]:
print(brain_anat)
explore_3D_array(brain_anat.numpy(), cmap=default_cmap)

# Building Scivol

It looks like ANTS already applies the affine (double check this), so I think you can just 86 that from your workflow

Initialize the scivol object

In [ ]:
skullstrip = Scivol("skullstrip")

Create the volumes:

In [ ]:
full_anat_vol = create_normalized_volume(raw_anat_img.numpy())
brain_anat_vol = create_normalized_volume(brain_anat.numpy())

Create the grids:

In [ ]:
explore_3D_array(full_anat_vol)
explore_3D_array(brain_anat_vol)

In [ ]:
skullstrip.add_grids([Grid("full_anat",[full_anat_vol]), Grid("brain_anat", [brain_anat_vol])])

In [ ]:
print(skullstrip)
skullstrip.save_scivol(output_folder)

Lets' quickly test this in our notebook

In [ ]:
test_compressed_file_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/skullstrip.scivol"
with gzip.open(test_compressed_file_path, 'r') as svin:
    scivol_bytes = svin.read()
scivol_str = scivol_bytes.decode('utf-8')

In [ ]:
scivol_dict = json.loads(scivol_str)

In [ ]:
print(type(scivol_dict))
print(scivol_dict['name'])
print([grid for grid in scivol_dict['grids']])
brain_anat_test_vol = np.asanyarray(scivol_dict['grids']['brain_anat']['frames'][0])
print(type(brain_anat_test_vol))
explore_3D_array(brain_anat_test_vol)

In [ ]:
explore_3D_array(np.asanyarray(scivol_dict['grids']['full_anat']['frames'][0]))

Before moving to blender to render these images, let's make sure that we can parse these scivol files here